# Selection: Simple

This is our introduction to the basic tidychef cell selection mechanisms.

## Source Data

The data source we're using for these examples is shown below:

| <span style="color:green">Note - this particular table has some very verbose headers we don't care about, so we'll be using `bounded=` to remove them from the previews as well as to show just the subset of data we're working with.</span>|
|-----------------------------------------|

The [full data source can be downloaded here](https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx). We'll be using th 8th tab named "Table 3a".

In [ ]:
from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx", tables="Table 3a")
preview(table, bounded="A4:H10")

## Row(s), Column(s) & Ranges

These are easier to show that tell, so we're going to give quick examples of each of these.


## Selecting a row

Lets just select a row based on its row mumber.

In [ ]:
row3 = table.row("7").label_as("Just row 7")
preview(row3, bounded="A5:H8")

## Selecting a row conditionally

We know we want one row and its a row with at least one cell with "Time period" in it.

In [ ]:
row_with_time_period = table.row_containing_strings(["Time period"]).label_as("A rows with 'Time period' in it")
preview(row_with_time_period, bounded="A4:H6")

Note - notice its strings plural not string singular, this allows us to be extra specific if we need to, example:

```
table.row_containing_strings(["Foo", "Bar", "Baz"])
```

This would still only return exactly one row, but only where said row contained a call ppulated with the value "Foo" and another call populated with the value "Bar" and another one populated with the word "Baz".

## Selecting a column

In [ ]:
column_c = table.column("C").label_as("I am column C")
preview(column_c, bounded="B2:D10")

## Selecting a column conditionally

In [ ]:
column = table.column_containing_strings(["Private new housing"])
preview(column_c, bounded="B2:D10") 

There's a point worth understanding in this example, the latter (find by expected contents) is typically more robust than the formcer (explciit reference) as its a good chance of contiuing to work even fi the layout shifts.

## Selecting a single cell conditionally

Very much what is sounds like, in this case you only pass one string because you're only even looking for one match.

In [ ]:
cell = table.cell_containing_string("MV5I").label_as("Single cell selection")
preview(cell, bounded="B4:D8") 

## Understanding `box_select()`

This is one of the less intuitive methods but also one you're gonna use a lot. The logic goes something like this:

- 1. For virtual all tabulated datasource the observations (the actual values) appear in a box or rectangular, shape - as shown below

In [ ]:
preview(table.excel_ref("B9:E12").label_as("Box like selection of value cells"), bounded="A5:E12")

* 2. The way you instinctively want to read the above blue selection **is the same way all human beings do**,starting from the top left of the blue box your eyes moves left to right, top to bottom - i.e the same way you'd read a book.

* 3. `.box_select()` mirrors this quirk of how we read, so it:
       - Starts with a single cell (the top-left part of the box)
       - Sweeps right gathering all the cells
       - Sweeps down gathering all cells beneath _those_ cells.

example follows:

In [ ]:
start_cell = table.excel_ref("B9")
box_of_cells = start_cell.box_select().label_as("Box like selection of value cells")
preview(box_of_cells, bounded="A5:E12")

...wait.... can't we just use that handy `excel_ref()` thing that keeps cropping up?

Yes you 100% can ..... but.... that's an explicit reference, so your code **breaks** the second someone moves whatever you're targetting so there's a trade off to consider.

Excel ref is convenient and very useful thing you'll almost certainly be making use of, but it (and all explicit references) carry an inherent fragility. Decided where to be explicit vs conditional with your selections is very context dependent but always something you should be considering when working with tidychef.

Speaking of.....

## Excel ref

This is analogous to when you click and drag on a spreadhseet to select a rand of cells, examples follow:

In [ ]:
preview(table.excel_ref("B9"), bounded="A5:E12")

In [ ]:
preview(table.excel_ref("B9:D11"), bounded="A5:E12")

In [ ]:
preview(table.excel_ref("8:10"), bounded="A5:E12")

In [ ]:
preview(table.excel_ref("D:E"), bounded="A5:E12")

## Is Numeric and Is Not Numeric

These common methods are wrappers on two very popular filters (more on filters later) and they work largely as you'd image.

In [ ]:
numeric = table.is_numeric().label_as("Is numeric")
preview(numeric, bounded="A5:E12")

In [ ]:
not_numeric = table.is_not_numeric().label_as("In not numeric")
preview(not_numeric, bounded="A5:E12")